In [1]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation
from collections import deque

puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv'
my_submission_path = 'submission.csv'

puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)

In [2]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_1/8']
# Two types: permutation & colors

,id,puzzle_type,solution_state,initial_state,num_wildcards
338,338,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,I;P;O;A;A;D;F;L;J;M;G;M;P;F;E;J;E;B;O;G;H;D;N;...,0
339,339,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,K;I;C;B;M;J;D;G;E;H;N;E;F;I;D;K;L;P;A;O;B;N;J;...,0
340,340,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,M;O;L;M;P;F;L;B;I;G;E;J;A;G;O;I;C;H;D;P;B;A;N;...,0
341,341,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,P;O;G;J;A;L;F;C;D;B;P;N;F;M;I;G;O;M;H;H;E;N;K;...,0
342,342,globe_1/8,A;A;C;C;E;E;G;G;I;I;K;K;M;M;O;O;B;B;D;D;F;F;H;...,F;H;O;M;C;J;K;I;J;L;B;N;B;G;K;H;D;P;A;D;E;N;P;...,2
343,343,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N21;N1;N9;N20;N30;N8;N25;N26;N11;N2;N16;N27;N4...,0
344,344,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N30;N0;N22;N15;N24;N19;N14;N18;N31;N6;N23;N16;...,0
345,345,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N19;N11;N22;N29;N15;N7;N14;N16;N0;N18;N9;N24;N...,0
346,346,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N9;N7;N17;N11;N30;N14;N5;N0;N16;N31;N6;N4;N1;N...,0
347,347,globe_1/8,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N26;N10;N17;N24;N11;N3;N14;N16;N23;N2;N21;N1;N...,0


In [3]:
idx = 343
N = 1+1
M = int(2*8)

In [14]:
test_globe = np.array([i for i in range(2*M)])
def print_globe(b):
    print(np.asarray(b).reshape(2, M))

r0_perm = np.array([i for i in range(2*M)])
r1_perm = np.array([i for i in range(2*M)])
f0_perm = np.array([i for i in range(2*M)])

r0_perm[M-1] = 0
for i in range(0, M-1):
    r0_perm[i] = i+1

r1_perm[2 * M - 1] = M
for i in range(M, 2*M-1):
    r1_perm[i] = i+1

for i in range(M//2):
    f0_perm[i] = M + M//2 - 1 - i

for i in range(M, M + M//2):
    f0_perm[i] = M//2-1 - (i-M)

nr0_perm = np.array([i for i in range(2*M)])
nr1_perm = np.array([i for i in range(2*M)])

for i in range(M-1):
    nr0_perm = nr0_perm[r0_perm]

for i in range(M-1):
    nr1_perm = nr1_perm[r1_perm]

In [15]:
all_moves = [r0_perm, nr0_perm, r1_perm, nr1_perm, f0_perm]
# all_moves = [r0_perm, r1_perm, f0_perm]

In [16]:
# 1e6 ~ 2 seconds
mx_mem = int(2e7)
mem_idx = 0

last_state = np.zeros(mx_mem, dtype=int)
last_move_mem = np.zeros(mx_mem, dtype=int)

print(last_state)

def dig_through_memory(idx):
    res = []
    while idx != 0:
        res.append(last_move_mem[idx])
        idx = last_state[idx]

    res = list(reversed(res))

    return res

@jit(nopython=True, fastmath = True)
def hash_perm(perm):
    base = 9973
    modb = 1000000007
    modc = 1000000009

    B, C = 0, 0
    for i in perm:
        B = (B * base) % modb + i
        C = (C * base) % modc + i

    return (B, C)

[0 0 0 ... 0 0 0]


In [17]:
def BFS():
    all_shorts = []
    
    mem_idx = 1
    initial_state = np.array([i for i in range(2*M)])
    visited = dict()
    visited[hash_perm(initial_state)] = 0

    Q = deque([(initial_state, 0, -1)])
    m = len(all_moves)

    while len(Q) > 0 and mem_idx + 10 < mx_mem:
        state, idx, last_move = Q.popleft()
        
        for j in range(m):

            if (last_move == 0 and j == 1) or (last_move == 1 and j == 0):
                continue

            if (last_move == 2 and j == 3) or (last_move == 3 and j == 2):
                continue

            if (last_move == 4 and j == 4):
                continue
            
            move = all_moves[j]
            next_state = state[move]
            next_hash = hash_perm(next_state)
            
            if(next_hash in visited):
                cur_moves = dig_through_memory(idx)
                shorter_path = dig_through_memory(visited[next_hash])
                cur_moves.append(j)
                all_shorts.append((cur_moves, shorter_path))
                continue
                
            visited[next_hash] = mem_idx
            
            last_state[mem_idx] = idx
            last_move_mem[mem_idx] = j
            
            mem_idx += 1

            Q.append((next_state, mem_idx, j))
        
    return all_shorts

In [18]:
%%time
a = BFS()

CPU times: user 1min 14s, sys: 1min 19s, total: 2min 34s
Wall time: 3min 7s


In [11]:
print(type(a))

<class 'list'>


In [12]:
b = 0
for i in a:
    b = max(len(i[0]) - len(i[1]), b)
    if len(i[0]) - len(i[1]) == 2:
        print(i)
        break

print(b)
#Doesn't work thus far

([1, 3, 1], [2])
2
